Read for more
https://github.com/yuwei2341/nlp/blob/master/cs224n/slides/6_Introduction_to_TensorFlow.pdf

# Concept
1. Big idea: express a numeric computation as a graph
    1. Nodes are operations which have any number of inputs and outputs
    2. Edges are tensors which flow between nodes
2. Nodes 
    1. Variable - Stateful nodes which output their current value - (weights) - operation y=x
    2. Placeholders - Nodes whose value is fed at execution time (inputs, labels, …)
    3. Math operations - Add, MatMul, ReLU, etc
3. Session
    1. A binding to a particular execution context (CPU, GPU, etc.)


In [14]:
import numpy as np
import tensorflow as tf

In [143]:
n_features = 20
n_batch = 10
n_y_dim = 2

In [144]:
# define a flow for 1-hidden-layer nn
b = tf.Variable(tf.zeros((n_y_dim,)))
W = tf.Variable(tf.random_uniform((n_features, n_y_dim), -1, 1))
x = tf.placeholder(tf.float32, shape=(n_batch, n_features), name='x')
h = tf.nn.relu(tf.matmul(x, W) + b)
y = tf.nn.softmax(h, name='y') 

label = tf.placeholder(tf.float32, [None, n_y_dim])
cross_entropy = -tf.reduce_mean(label * tf.log(y), reduction_indices=[1])

lr = 0.01
train_step = tf.train.GradientDescentOptimizer(lr).minimize(cross_entropy)

In [145]:
# print(tf.get_default_graph().get_operations()) 

In [147]:
WW = np.random.uniform(size=(n_features, n_y_dim))
def gen_dataset():
    x0 = np.linspace(0, 10, n_features)
    x_batch = np.array([x0 + np.random.randn(n_features) for i in range(n_batch)])
    label_batch = np.matmul(x_batch, WW) + np.random.randn(n_y_dim)
    return x_batch, label_batch

batch_x, batch_label = gen_dataset()
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(10):        
        ce_val, _ = sess.run([cross_entropy, train_step], feed_dict={x: batch_x, label: batch_label})
        print('loss (CE): {:.1f}'.format(ce_val.mean()))

loss (CE): 42.8
loss (CE): 33.3
loss (CE): 33.3
loss (CE): 33.3
loss (CE): 33.3
loss (CE): 33.3
loss (CE): 33.3
loss (CE): 33.3
loss (CE): 33.3
loss (CE): 33.3


#### Run tensorboard
tensorboard --logdir="./graphs" --port 6006

In [150]:
a = tf.constant(2,)
b = tf.constant(3,)
x = tf.add(a, b,)
writer = tf.summary.FileWriter('./graphs', tf.get_default_graph())
with tf.Session() as sess:
     # writer = tf.summary.FileWriter('./graphs', sess.graph)
    print(sess.run(x))
writer.close()

5


In [ ]:
import os
os.system('tensorboard --logdir="./graphs" --port 6006')

variable scope: namespace to avoid crash;

In [51]:
with tf.variable_scope('foo'):
    v = tf.get_variable('v', shape=[1])

with tf.variable_scope('foo', reuse=True):
    v1 = tf.get_variable('v') # Shared variable
    
with tf.variable_scope('foo', reuse=False):
    v1 = tf.get_variable('v') # CRASH: Variable foo/v already exists